In [1]:
using TuringModels

Turing.setadbackend(:reverse_diff)
#nbTuring.turnprogress(false);

d = CSV.read(rel_path("..", "data", "UCBadmit.csv"), delim=';');
size(d) # Should be 12x5

(12, 5)

Turing model

In [2]:
@model m11_5(admit, applications) = begin
    N=length(applications)
    θ ~ Truncated(Exponential(1), 0, Inf)
    α ~ Normal(0,2)

    for i ∈ 1:N
        prob = logistic(α)
        # alpha and beta for the BetaBinomial must be provided.
        # The two parameterizations are related by
        # alpha = prob * theta, and beta = (1-prob) * theta.
        # See https://github.com/rmcelreath/rethinking/blob/master/man/dbetabinom.Rd
        alpha = prob * θ
        beta = (1 - prob) * θ

        admit[i] ~ BetaBinomial(applications[i], alpha, beta)
    end
end

m11_5 (generic function with 3 methods)

Sample

In [3]:
posterior = sample(m11_5(d[:admit],d[:applications]), Turing.NUTS(4000, 1000, 0.9));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.8
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/support/hmc_core.jl:235
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:169
┌ Info:  Adapted ϵ = 0.17662275392141913, std = [1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 21.64463686100001;
  #lf / sample        = 0.0;
  #evals / sa

Draw summary

In [4]:
describe(posterior)

Iterations = 1:4000
Thinning interval = 1
Chains = 1
Samples per chain = 4000

Empirical Posterior Estimates:
              Mean           SD        Naive SE        MCSE         ESS   
       α  -0.3680032711 0.308081635 0.00487119836 0.00629685013 2393.78371
  lf_num   0.0000000000 0.000000000 0.00000000000 0.00000000000        NaN
 elapsed   0.0054111592 0.012700305 0.00020080945 0.00026146335 2359.42767
 epsilon   0.1994558359 0.108892744 0.00172174546 0.00641234320  288.37949
      lp -71.0967041477 1.603959239 0.02536082235 0.05371726739  891.57695
eval_num  16.9715000000 7.793193904 0.12322121491 0.25453238925  937.44297
       θ   2.7519568422 1.112317124 0.01758727796 0.03059945495 1321.38650
  lf_eps   0.1994558359 0.108892744 0.00172174546 0.00641234320  288.37949

Quantiles:
               2.5%           25.0%         50.0%         75.0%         97.5%    
       α  -0.95568761507  -0.5694470385  -0.368913475  -0.169150708   0.240441826
  lf_num   0.00000000000   0.0000000000

Result rethinking

In [5]:
m115rethinking = "
         mean   sd  5.5% 94.5% n_eff Rhat
theta  2.74 0.96  1.43  4.37  3583    1
a       -0.37 0.31 -0.87  0.12  3210    1
";

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*